<a href="https://colab.research.google.com/github/jasmeen2105/Pneumonia_Detection/blob/main/Pneumonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!mkdir -p ~/.kaggle

In [27]:
!mv kaggle.json ~/.kaggle/

mv: cannot stat 'kaggle.json': No such file or directory


In [28]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [29]:
!kaggle datasets download -d [/content/kaggle (1).json]

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `kaggle datasets download -d [/content/kaggle (1).json]'


In [30]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [31]:
import zipfile

with zipfile.ZipFile("chest-xray-pneumonia.zip", "r") as zip_ref:
    zip_ref.extractall("chest_xray")

In [32]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [33]:
# Define dataset path
data_dir = '/content/chest_xray/chest_xray'
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
val_dir = os.path.join(data_dir, "val")

In [34]:
# Image size & batch size
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

In [35]:
# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip=True)

train_generator = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
test_generator = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_generator = datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [36]:
# CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [37]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [38]:
# Train the model
epochs = 10
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 333s 2s/step - accuracy: 0.7703 - loss: 0.5001 - val_accuracy: 0.6250 - val_loss: 0.8163
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9012 - loss: 0.2468 - val_accuracy: 0.7500 - val_loss: 1.0037
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.9113 - loss: 0.2085 - val_accuracy: 0.5625 - val_loss: 1.3727
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9295 - loss: 0.1823 - val_accuracy: 0.5625 - val_loss: 1.3712
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 381s 2s/step - accuracy: 0.9250 - loss: 0.1954 - val_accuracy: 0.6875 - val_loss: 1.4988
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 378s 2s/step - accuracy: 0.9274 - loss: 0.1737 - val_accuracy: 0.6250 - val_loss: 1.5264
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.9346 - loss: 0.1783 - val_accuracy: 0.6250 - val_loss: 1.1093
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.9392 - loss: 0.1653 - val_accu

In [39]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 759ms/step - accuracy: 0.9066 - loss: 0.2983
Test Accuracy: 88.78%


In [40]:
# Function to predict pneumonia
def predict_pneumonia(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Reshape for model
    prediction = model.predict(img)[0][0]
    return "Pneumonia Detected" if prediction > 0.5 else "Normal"

In [41]:
# Test prediction
sample_image = "/content/chest_xray/chest_xray/test/NORMAL/IM-0006-0001.jpeg"
print(predict_pneumonia(sample_image))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Normal
